In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import OUTPUT_PATH, Versions

groups = [Versions.LTS, Versions.REGULAR, Versions.REV]
# groups = [Versions.TEST]

results = []
for group in groups:
    results.append(group.diff_pairs(result_path=OUTPUT_PATH / "src" / group.name))

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.
[     versions.py:127] INFO: Diffing LTS
[     versions.py:132] INFO: Comparing 4.4.0-21-generic-amd64 and 4.15.0-20-generic-amd64 to /Users/szhong/Downloads/bpf-study/output/src/LTS/4.4.0-21-generic-amd64_4.15.0-20-generic-amd64
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.4.0-21-generic-amd64.pkl
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.15.0-20-generic-amd64.pkl
[     versions.py:132] INFO: Comparing 4.15.0-20-generic-amd64 and 5.4.0-26-generic-amd64 to /Users/szhong/Downloads/bpf-study/output/src/LTS/4.15.0-20-generic-amd64_5.4.0-26-generic-amd64
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.4.0-26-generic-amd64.pkl
[     versions.py:132] INFO: Comparing 5.4.0-26-generic-amd64 and 5.15.0-25-generic-amd64 to /Users/szhong/Downloads/bpf-study/output/src/LTS/5.4.0-26-gene

In [3]:
import pandas as pd
from depsurf.output import save_df

df = pd.concat(results, axis=1)
df = df.loc[(df != 0).any(axis=1)]
df = df.fillna(0).astype(int)
save_df(df, "src/summary")

[           pd.py:24 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/src/summary.txt
[           pd.py:33 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/src/summary.pkl


LTS                                                                                                                                                                                                Regular                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               \
                               (4.4.0-21-generic-amd64, 4.15.0-20-generic-amd64) (4.15.0-20-generic-amd64, 5.4.0-26-generic-amd64) (5.4.0-26-generic-amd64, 5.15.0-25-generic-amd64) (5.15.0-25-generic-amd64, 6.8.0-22-generic-amd64) (4.4.0-21-generic-amd64, 4.8.0-22-generic-amd64) (4.8.0-22-generic-amd64, 4.10.0-19-generic-amd64) (4.10.0-19-generic-amd64, 4.13.0-16-generic-amd64) (4.13.0-16-generic-amd64, 4.15.0-20-generic-amd64) (4.15.0-20-generic-amd64, 4.18.0-10-generic-amd64) (4.18.0-10-generic-amd64, 5.0.0-13-generic-amd64) (5.0.0-13-generic-amd64, 5.3.0-18-generic-amd64) (5.3.0-18-generic-amd64, 5.4.0-26-generic-amd64) (5.4.0-26-generic-amd64, 5.8.0-25-generic-amd64) (5.8.0-25-generic-amd64, 5.11.0-16-generic-amd64) (5.11.0-16-generic-amd64, 5.13.0-19-generic-amd64) (5.13.0-19-generic-amd64, 5.15.0-25-generic-amd64) (5.15.0-25-generic-amd64, 5.19.0-21-generic-amd64) (5.19.0-21-generic-amd64, 6.2.0-20-generic-amd64) (6.2.0-20-generic-amd64, 6.5.0-9-generic-amd64)   
Function   Old                                                             36309                                             42299                                             48018                                             53956                                            36309                                             38211                                              39148                                              40741                                              42299                                             44488                                            45277                                            46992                                            48018                                             51936                                              54302                                              53193                                              53956                                             55843                                           56978   
           New                                                             42299                                             48018                                             53956                                             61966                                            38211                                             39148                                              40741                                              42299                                              44488                                             45277                                            46992                                            48018                                            51936                                             54302                                              53193                                              53956                                              55843                                             56978                                           59365   
           Added                                                            8600                                              8929           